In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
client

MongoClient()

Aggregation Examples

MongoDB에서 집계를 수행하는 방법에는 여러 가지가 있습니다. 이 예제는 map reduce를 사용하고 group 메소드를 사용하는 새로운 집계 프레임 워크를 다룹니다.

Setup

시작하려면 Aggregation를 수행 할 수 있는 몇 가지 예제 데이터를 삽입 합니다.

In [3]:
from pymongo import MongoClient
db = MongoClient().aggregation_example
db = client.aggregation_example
result = db.things.insert_many([
    {'x' : 1, 'tags': ['dogs', 'cat']},
    {'x' : 2, 'tags': ['cat']},
    {'x' : 2, 'tags': ['mouse', 'cat', 'dog']},
    {'x' : 3, 'tags': []}
])
result.inserted_ids

[ObjectId('5fcee43e9d5495708c99dcfd'),
 ObjectId('5fcee43e9d5495708c99dcfe'),
 ObjectId('5fcee43e9d5495708c99dcff'),
 ObjectId('5fcee43e9d5495708c99dd00')]

Aggregation 프레임 워크

이 예제에서는 Aggregation 프레임 워크를 사용하기 위해 aggregate() 메서드를 사용하는 방법을 보여줍니다. 전체 컬렉션에서 태그 배열의 각 태그에 대한 발생 수를 계산하기 위해 간단한 Aggregation를 수행합니다. 이를 달성하려면 파이프 라인에 세 가지 작업을 전달해야 합니다. 먼저 태그 배열을 풀고 태그 별로 그룹화하고 합산 한 다음 마지막으로 개수별로 정렬 해야 합니다.

파이썬 dictionary는 순서를 유지하지 않으므로 명시적인 순서가 필요한 경우 SON 또는 collections.OrderedDict를 사용해야 합니다(예:"$ sort").

In [4]:
from bson.son import SON
pipeline = [
    {'$unwind': '$tags'},
    {'$group': {'_id': '$tags', 'count': {'$sum': 1}}},
    {'$sort': SON([('count', -1), ('_id', -1)])}
]

import pprint
pprint.pprint(list(db.things.aggregate(pipeline)))

[{'_id': 'cat', 'count': 6},
 {'_id': 'mouse', 'count': 2},
 {'_id': 'dogs', 'count': 2},
 {'_id': 'dog', 'count': 2}]


이 Aggregation에 대한 explain plan을 실행하려면 command() 메소드를 사용하십시오

In [5]:
db.command('aggregate', 'things', pipeline=pipeline, explain=True)

{'stages': [{'$cursor': {'queryPlanner': {'plannerVersion': 1,
     'namespace': 'aggregation_example.things',
     'indexFilterSet': False,
     'parsedQuery': {},
     'queryHash': 'EE638FFC',
     'planCacheKey': 'EE638FFC',
     'winningPlan': {'stage': 'PROJECTION_SIMPLE',
      'transformBy': {'tags': 1, '_id': 0},
      'inputStage': {'stage': 'COLLSCAN', 'direction': 'forward'}},
     'rejectedPlans': []}}},
  {'$unwind': {'path': '$tags'}},
  {'$group': {'_id': '$tags', 'count': {'$sum': {'$const': 1}}}},
  {'$sort': {'sortKey': {'count': -1, '_id': -1}}}],
 'serverInfo': {'host': '8c3c1ba04536',
  'port': 27017,
  'version': '4.4.1',
  'gitVersion': 'ad91a93a5a31e175f5cbf8c69561e788bbc55ce1'},
 'ok': 1.0}

단순한 Aggregation뿐만 아니라 Aggregation 프레임워크는 반환된 데이터를 재구성하는 프로젝션 기능을 제공합니다. 프로젝션 및 Aggregation를 사용하여 계산된 필드를 추가하고, 새 가상 하위 개체를 만들고, 하위 필드를 결과의 최상위 수준으로 추출할 수 있습니다.

Map / Reduce

Aggregation를 위한 또 다른 옵션은 Map/Reduce프레임워크를 사용하는 것입니다. 여기서는 전체 컬렉션에서 태그 배열의 각 태그에 대한 발생 수를 계산하는 맵을 정의하고 함수를 줄입니다.

map 함수는 배열의 각 태그에 대해 단일 (key, 1) 쌍을 내 보냅니다.

In [6]:
from bson.code import Code
mapper = Code("""
                function () {
                    this.tags.forEach(function(z) {
                        emit(z, 1);
                    });
                }
                """)

reduce 함수는 주어진 키에 대해 내보낸 모든 값을 합산합니다.

In [7]:
reducer = Code("""
                function (key, values) {
                    var total = 0;
                    for (var i = 0; i < values.length; i++) {
                        total += values[i];
                    }
                    return total;
                }
                """)

reduce 함수는 다른 reduce 단계의 결과에 대해 반복적으로 호출될 수 있으므로 values.length를 반환 할 수 없습니다.

In [8]:
result = db.things.map_reduce(mapper, reducer, 'myresults')
for doc in result.find().sort('_id'):
    pprint.pprint(doc)

{'_id': 'cat', 'value': 6.0}
{'_id': 'dog', 'value': 2.0}
{'_id': 'dogs', 'value': 2.0}
{'_id': 'mouse', 'value': 2.0}


고급 Map/Reduce

PyMongo의 API는 MongoDB의 Map/Reduce 엔진의 모든 기능을 지원합니다. 한 가지 흥미로운 기능은 full_response=True 를 map_reduce()에 전달하여 원하는 경우 더 자세한 결과를 얻을 수 있다는 것 입니다. 이렇게 하면 결과 컬렉션이 아닌 Map/Reduce 명령에 대한 전체 응답이 반환됩니다.

In [9]:
pprint.pprint(
    db.things.map_reduce(mapper, reducer, 'myresults', full_response=True))

{'ok': 1.0, 'result': 'myresults'}


모든 선택적 map/reduce 매개 변수도 지원되며 키워드 인수로 전달하기만 하면 됩니다. 이 예에서는 쿼리 매개 변수를 사용하여 매핑될 문서를 제한합니다.

In [10]:
results = db.things.map_reduce(
    mapper, reducer, 'myresults', query={'x': {'$lt': 2}})
for doc in results.find().sort('_id'):
    pprint.pprint(doc)

{'_id': 'cat', 'value': 2.0}
{'_id': 'dogs', 'value': 2.0}


SON 또는 collections.OrderedDict를 사용하여 결과 컬렉션을 저장할 다른 데이터베이스를 지정할 수 있습니다.

In [11]:
from bson.son import SON
pprint.pprint(
    db.things.map_reduce(
        mapper, 
        reducer,
        out=SON([('replace', 'results'), ('db', 'outdb')]),
        full_response=True))

{'ok': 1.0, 'result': {'collection': 'results', 'db': 'outdb'}}
